# A python Notebook to scrap data from a website

Import the necessary libraries

In [ ]:
import requests
import bs4
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


> with webdriver_manager it automatically downloads the latest version of the driver

In [7]:
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service)

In [8]:
# set the url to extract the data

url = 'https://www.barchart.com/futures/major-commodities'

Method 1: request and bs4

In [12]:
# scrape the data from the website using requests and beautifulsoup
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=  headers)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<template id="grid-styles">
 <link href="https://assets.barchart.com/fonts/barfont.1575916156.css?4s3ylr" rel="stylesheet" type="text/css"/>
 <link href="https://assets.barchart.com/build/datatable-QA4XNBQ5.css" id="sheet" rel="stylesheet"/>
</template>
<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en">
 <head>
  <!-- OneTrust Cookies Consent Notice start for barchart.com -->
  <script charset="UTF-8" data-domain-script="5ccd8ca4-7e70-4581-8b5d-149aef8c31d6" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   function OptanonWrapper() { }
  </script>
  <!-- OneTrust Cookies Consent Notice end for barchart.com -->
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <base href="/"/>
  <title>
   Commodities Prices - Barchart.com
  </title>
  <meta content="Free Major Commodities futures pri

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
def get_soup(url):
    response = requests.get(url, headers=headers)
    return bs4.BeautifulSoup(response.text, 'html.parser')

def scrape_data(url):
    soup = get_soup(url)
    print(f"Scraping data from {url}")
    print(soup.prettify())

    # Find all links on the page
    links = soup.find_all('a', href=True)
    for link in links:
        sub_url = urljoin(url, link['href'])
        if sub_url.startswith(url):  # Ensure the link is within the same domain
            scrape_data(sub_url)

# Start scraping from the main URL
scrape_data(url)

Method 2: Selenium

In [ ]:
try:
    driver.get(url)

    # Wait for the table element to load 
    wait = WebDriverWait(driver, 15)
    table_element = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table"))
    )

    # Find the table rows
    rows = table_element.find_elements(By.TAG_NAME, "tr")

    # Iterate over rows and print each row's cells (both headers and data cells)
    for row in rows:
        # Try to get header cells first, if available, otherwise use data cells
        headers = row.find_elements(By.TAG_NAME, "th")
        cells = row.find_elements(By.TAG_NAME, "td")
        if headers:
            cell_texts = [header.text for header in headers]
        else:
            cell_texts = [cell.text for cell in cells]
        # Print the row if it contains text
        if cell_texts:
            print(cell_texts)
finally:
    driver.quit()